<a href="https://colab.research.google.com/github/sharjeel-png/Elevate_Labs-Project_E-commerce-Return-Rate-Reduction-Analysis/blob/main/Elevate_Labs_data_analyst_project_Logistic_Regression_22_12_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Cell 1 — Install required library**

In [1]:
# Cell 1

# Install gdown to automatically download files from Google Drive
!pip install gdown
!pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 8.3 MB/s eta 0:00:00


**Cell 2 — Import required libraries**

In [2]:
# Cell 2

import pandas as pd   # pandas → data manipulation
import numpy as np    # numpy → numerical operations
import gdown          # gdown → download dataset automatically


**Cell 3 — Download dataset automatically using gdown**

In [3]:
# Cell 3

# global_superstore_2016.xlsx stored on google drive
# https://docs.google.com/spreadsheets/d/1n5Df1b1Og0OU5k5ixU8zThtwxsmP4snc/edit?usp=sharing&ouid=111322316490153916734&rtpof=true&sd=true

file_id = "1n5Df1b1Og0OU5k5ixU8zThtwxsmP4snc"
url = f"https://drive.google.com/uc?id={file_id}"

gdown.download(url, "global_superstore_2016.xlsx", quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1n5Df1b1Og0OU5k5ixU8zThtwxsmP4snc
To: /content/global_superstore_2016.xlsx
100%|██████████| 8.50M/8.50M [00:00<00:00, 63.7MB/s]


'global_superstore_2016.xlsx'

**Cell 4 — Load Orders and Returns sheets**

In [4]:
# Cell 4

# Load the Orders sheet (main transactional data)
orders = pd.read_excel("global_superstore_2016.xlsx", sheet_name="Orders")

# Load the Returns sheet (contains only returned orders)
returns = pd.read_excel("global_superstore_2016.xlsx", sheet_name="Returns")


**Cell 5 — Create target variable (Returned: 1 / 0)**

In [5]:
# Cell 5

# Mark all orders present in Returns sheet as Returned = 1
returns["Returned"] = 1

# Keep only Order ID and Returned flag
returns = returns[["Order ID", "Returned"]]

# Merge Orders with Returns using Order ID
# Left join ensures all orders are kept
df = orders.merge(
    returns,
    on="Order ID",
    how="left"
)

# Orders not found in Returns sheet are not returned → mark as 0
df["Returned"] = df["Returned"].fillna(0)


**Cell 6 — Sanity check for target variable**

In [6]:
# Cell 6

# Check distribution of returned vs non-returned orders
df["Returned"].value_counts()


,count
Returned,
0.0,49070
1.0,2220


**Cell 7 — Select features and target**

In [7]:
# Cell 7

# These features influence return behavior
features = [
    "Category",
    "Sub-Category",
    "Segment",
    "Market",
    "Ship Mode",
    "Discount",
    "Quantity",
    "Sales"
]

# X → input features
# y → target variable
X = df[features]
y = df["Returned"]


**Cell 8 — Encode categorical variables**

In [8]:
# Cell 8

# Convert text columns into numerical form using one-hot encoding
# drop_first=True avoids dummy variable trap
X = pd.get_dummies(X, drop_first=True)


**Cell 9 — Split data into train and test sets**

In [9]:
# Cell 9

from sklearn.model_selection import train_test_split

# 80% training data, 20% testing data
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)


**Cell 10 — Train Logistic Regression model**

In [10]:
# Cell 10

from sklearn.linear_model import LogisticRegression

# Logistic Regression is suitable for binary outcomes (Yes/No)
model = LogisticRegression(max_iter=10000000)

# Train the model on training data
model.fit(X_train, y_train)


LogisticRegression(max_iter=10000000)

**Cell 11 — Generate return risk score**

In [11]:
# Cell 11

# Predict probability of return for each order
# [:, 1] extracts probability of class "Returned = 1"
df["Return_Risk_Score"] = model.predict_proba(X)[:, 1]


**Cell 12 — Aggregate risk at PRODUCT level**

In [12]:
# Cell 12

# Group by product to get product-level risk insights
product_risk = (
    df.groupby(["Product Name", "Category", "Sub-Category"])
    .agg(
        Total_Orders=("Order ID", "nunique"),        # Unique orders
        Total_Returns=("Returned", "sum"),           # Total returned orders
        Avg_Risk_Score=("Return_Risk_Score", "mean") # Average risk score
    )
    .reset_index()
)


**Cell 13 — Create business-friendly risk labels**

In [13]:
# Cell 13

# Convert numeric risk score into business-friendly labels
def risk_label(score):
    if score >= 0.6:
        return "High"
    elif score >= 0.3:
        return "Medium"
    else:
        return "Low"

product_risk["Risk_Level"] = product_risk["Avg_Risk_Score"].apply(risk_label)

product_risk.head()

,Product Name,Category,Sub-Category,Total_Orders,Total_Returns,Avg_Risk_Score,Risk_Level
0,"""While you Were Out"" Message Book, One Form pe...",Office Supplies,Paper,3,0.0,0.047220,Low
1,"#10 Gummed Flap White Envelopes, 100/Box",Office Supplies,Envelopes,4,1.0,0.048960,Low
2,#10 Self-Seal White Envelopes,Office Supplies,Envelopes,4,0.0,0.045460,Low
3,"#10 White Business Envelopes,4 1/8 x 9 1/2",Office Supplies,Envelopes,7,0.0,0.044065,Low
4,"#10- 4 1/8"" x 9 1/2"" Recycled Envelopes",Office Supplies,Envelopes,10,0.0,0.046730,Low


**Cell 14 — Export final CSV (deliverable)**

In [14]:
# Cell 14

# Export high-risk products dataset
product_risk.to_csv("high_risk_products.csv", index=False)
